# Lab 4 Individual Analysis - Muon Lifetime

This exercise will continue with the problem of calibration curves and data transformations.  You will also expand your use of the **lmfit** package.

In [ ]:
import numpy as np  # import the numpy library functions.
import scipy.constants as const # import physical constants
import matplotlib.pyplot as plt # plotting functions
import uncertainties as unc # Uncertainties package.  Good for simple error propagation
# directive below puts the plots in the notebook
%matplotlib inline

## The calibration curve



### Enter the calibration data

Analyze the calibration data of TAC peaks that come from the Digital Delay Generator.  Then enter these results along with the known time values of 1.00, 2.00, ..., 9.00 microseconds to create two arrays: `cal_channels` and `cal_times`.

In [ ]:
# Channels vs. time in microseconds from TAC calibration.

cal_channels = 
cal_times =  # microseconds


### Fit and extract a calibration

Use LMFIT to fit the calibration data.  Obtain parameters that will allow you to convert a channel into a time in microseconds.

### Save the fit parameters

Save the fit parameters and their `stderr`s as `uncertainties` objects.  This will allow you to easily propagate the uncertainties in the final values.

In [ ]:
# Save fit parameters as uncertainty objects


## Muon lifetime data, first pass

Import the muon lifetime data from the .mca file.  Use the Python package `mcareader` to do this.  To access the mcareader package, open a terminal window and type

`pip install mcareader`

You only need to do this once.  If you get an error when you try to import `mcareader`, you do not have it installed.

The cell below shows what using mcareader looks like, using the calibration data.  Note the use of the `Mca()` function and the return structures attributes.

For an example of how to use this small Python package, see the [mcareader documentation](https://github.com/Dih5/mcareader/blob/master/demos/intro.ipynb)

In [ ]:
import mcareader as mca

TAC_cal = mca.Mca('MuLife_calibration_data.mca')

### Ignore warnings about missing calibration data.  ###

(TAC_x, TAC_y) = TAC_cal.get_points(trim_zeros=False)

plt.plot(TAC_x, TAC_y, '-b');

### Your turn: import and plot the muon lifetime data

Use what you learned in previous notebooks to plot the raw data set.  Remember to label the axes, etc.

In [ ]:
# Import first


In [ ]:
# Then plot



### Remove the bad data

Study the data set and extract the "good" part of the data, above the spike at around channel 45.  Use a Python "slice" to do this, and replot it to make sure you have a decent set of trimmed data.

## Data fitting, part 1
Fit the trimmed data to an exponential decay plus a constant background term:

$$ N(x) = Ae^{-x/\tau} + C $$

producing parameters $A$, $C$, and $\tau$ in channel units.

Do this by using the LMFIT built in models to create a "composite" model with `ExponentialModel` plus `ConstantModel`.  See LMFIT [Built-in Fitting Models](https://lmfit.github.io/lmfit-py/builtin_models.html) for an example that combines a peak model with a background model.

You will need to set the starting values of the `amplitude` ($A$), the `decay` ($\tau$) and the background constant `c` ($C$).  The `guess()` method to set the parameters will not work on composite models.  These are not hard to estimate from the data plot. Look at the plot to estimate staring values for `c`, `decay`, and `amplitude`.  Call them `start_c`, `start_ampl`, and `start_decay`. Then you can call the fit routine with

`model_fit = model.fit(trim_y, c=start_c, amplitude=start_ampl, decay=start_decay, x=trim_x)`


### Replot, with a fitline

Use the `eval()` function to msake a fit line, and plot it with the data.

### Calculate the lifetime

Combine the fit results and the calibration constants along with their uncertainties to calculate a value for the lifetime and background term, including uncertainties.  Print these out with correct significant figures.  (Use the `uncertainties` package.) 

## Data fitting part 1-A

Redo the fit above, but include weights for the fitting routine.  The weights should be an array, with each element being $1/\sigma_i$, where $\sigma_i$ is the uncertainty in the point.

Because this is an experiment based on random statitistics, the uncertainty in any channel is the square root of the number of counts in that channel.  This is a consequence of Poisson statistics.  So the weights array can be specified as `weights = 1.0/np.sqrt(trim_y)` where `trim_y` is the array of trimmed chanel counts.

NOTE: If you get an error about dividing by zero, either make sure you have no zeros in your data set, or add the following argument to the `fit()` function: `nan_policy='omit'`.

### Recalculate the lifetime, from the results of the weighted fit.

Redo what you did above to obtain a new version of the lifetime, based on the weighted fit.

### Compare version 1 and version 1-A

Look at the `fit_report()` and comment below about the $\chi^2$ (chi-square) statistics for each version of the fit: Note the differences, and explain them.

### $\rightarrow$ Click on this to open a cell to write your discussion $\leftarrow$ 

## Data fitting, part 2

Based on the record of **C-Singles**, **Valid Starts**, **Valid Conversions**, and **Elapsed Time**, your group should calculate an expected background level, per channel, for the experiment. 

Then, use that estimate in the fit as follows.  Obtain the **Parameters** structure from the previous fit, and for marameter `c` (the background constant), set its `vary` attribute to `False`, and then its `value` attribute to your expected background level.  

Feed the revised parameters into the `fit()` function, and redo the fit.  (You may leave the weight in place, or not, your choice).

Scroll to the bottom of [Getting started with Non-Linear Least-Squares Fitting](https://lmfit.github.io/lmfit-py/intro.html) to see an example of controlling the parameters.

In [ ]:
# Obtain the current fit parameters
params = model_fit.params

# You do the rest


### Compare the fit curves

Create another fitline, based on the fit result with the fixed background term, and plot it over the data, along with the original fitline.

Your group will want to discuss these in the group document.

### Calculate the lifetime (again)

From the fit, calculate another value for the lifetime (with uncertainty).

## Data fitting, part 3

Here, you want to manipulate the data set to create a new set of data, by "coarse binning".  This means to combine the counts from adajacent channels into a single new channel.  For example, with 1024 channels, one could "rebin" by summing channels 0-15 into one bin, 16-31 in the next, 32-47 in the third, etc.  This would reduce the data set from 1024 channels to 64 channels.  By adding in this way, the scatter of the data set is reduced.  One must also adjust the `x` or channel values similarly, otherwise, the data set is scaled horizontally, and the channel-time calibration will not work.

In the following, you will write a function to rebin the data by summing over successive channels and rescaling the channel numbers.  Then you will plot and fit the results, as before, and see whether this improves anything.

### Write a function `coarse_bin()`

Define a function with the folowing specifications:

* Arguments:
  * **x** : array of *x* or channel numbers
  * **y** : array of *y* or count numbers
  * **binwidth** : the width of the new bins, or number of channels to sum together

* Returns:
  * **new_x, new_y**: A tuple of Numpy arrays of the average values of <i>x</i> and sum of values of <i>y</i>, across the number of summed channels.

In [ ]:

def coarse_bin(x, y, binwidth=None):
    ###
    # You fill in the body of the function
    ###
   
    return (np.array(new_x),np.array(new_y))

### Apply your function

Test your function, and see that it works correctly.  Then plot the rebinned data.

In [ ]:
newx, newy = coarse_bin(trim_x,trim_y,binwidth=16)

### Fit it

Use the plot you made to estimate the starting values of the parameters for the rebinned data set.  Then fit it and extract a value for the lifetime, as you did above.

### Plot the rebinned data and the fit

If you have not done so already, plot the rebinned data and its fit curve.

### Calculate lifetime and background with rebinned data

## Muon lifetime calculation summary
Make a table with the muon lifetimes computed in steps 1, 1a, 2 and 3.
Calculated value of muon lifetime and background rate, including uncertainty.